###  Pasos a seguir en el proceso de 'scraping':

1. Encuentra la URL que quieres 'escrapear'.
2. Inspecciona la página (código fuente).
3. Localiza los datos que necesitas obtener.
4. Desarrolla tu código en Python.
    1. Crea tu sopa
    2. Busca los elementos que cotienen los datos y extráelos
5. Ejecuta tu código y obten los datos.
6. Almacena los datos en el formato requerido.

In [1]:
# Importamos librerías
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

## Caso 1: Scraping de un catálogo: Labirratorium

In [2]:
URL = "https://www.labirratorium.com/es/67-cervezas-por-estilo?page="

Queremos obtener un dataFrame con todas las cervezas del catálogo y sus características descritas. Analizamos la página para ver qué tenemos que hacer para conseguirlo

In [3]:
# La web tiene 49 páginas con 12 cervezas listadas en cada página.

Hacemos la consulta (request) y creamos la SOPA inicial:

In [4]:
web_html = requests.get(URL).content
soup = bs(web_html, "lxml")
type(soup)

bs4.BeautifulSoup

In [7]:
ls_href_tags = soup.find_all("a", class_="thumbnail product-thumbnail")
ls_hrefs = [tag["href"] for tag in ls_href_tags]
# ls_hrefs = []
# for tag in ls_href_tags:
#     ls_hrefs.append(tag["href"])

ls_hrefs


['https://www.labirratorium.com/es/inicio/831--vikingathor-.html',
 'https://www.labirratorium.com/es/cerveza-lambic/284-boon-kriek-2013.html',
 'https://www.labirratorium.com/es/cervezas-alemanas/225-stortebeker-schwarz-bier.html',
 'https://www.labirratorium.com/es/botella-33cl/199-orval.html',
 'https://www.labirratorium.com/es/cervezas-alemanas/184-augustiner-lagerbier-hell.html',
 'https://www.labirratorium.com/es/inicio/183-schneider-eisbock.html',
 'https://www.labirratorium.com/es/inicio/181-schlenkerla-rauchbier-weizen.html',
 'https://www.labirratorium.com/es/inicio/173-samuel-adams-boston-lager.html',
 'https://www.labirratorium.com/es/inicio/82-westmalle-dubbel.html',
 'https://www.labirratorium.com/es/inicio/75-duchesse-de-bourgogne.html',
 'https://www.labirratorium.com/es/inicio/61-tripel-karmeliet33.html',
 'https://www.labirratorium.com/es/inicio/21-weihenstephaner-vitus.html']

Nombre,	precio,	descripcion corta, descripción larga, ruta imagen, marca, barcode,	Estilo, Origen, % Alc., % Alc. Exacto, Volumen (Cl), Tipo Fermentación, IBU, IBU Exacto, Color, Envase

In [8]:
first_url = ls_hrefs[0]
first_data = requests.get(first_url).content
first_soup = bs(first_data, "lxml")

In [13]:
def get_beer(url_beer):    
    web_html = requests.get(url_beer).content
    first_soup = bs(web_html, "lxml")


    first_name = first_soup.find("h1", class_="h1 product-detail-name").string
    # print(first_name)

    div_main_info = first_soup.find("div", class_= "col-form_id-form_15874367062488778 col-md-6 col-lg-6 col-xl-6")
    first_price = div_main_info.find("span", itemprop="price")["content"]
    # print(first_price)

    first_short_description = div_main_info.find("div", itemprop="description").p.string
    # print(first_short_description)

    div_img_cover = first_soup.find("div", class_="product-cover")
    first_img = div_img_cover.find("img", itemprop="image")["src"]
    # print(first_img)

    first_long_description = first_soup.find("div", class_="product-description").text
    # print(first_long_description)

    data_sheet = first_soup.find("dl", class_="data-sheet")
    # first_estilo=

    ls_names = [tag.string for tag in data_sheet.find_all("dt")]
    ls_values = [tag.string for tag in data_sheet.find_all("dd")]

    first_d_props = dict(zip(ls_names, ls_values))

    first_d_total = {"Nombre": first_name, 
                    "Precio": first_price, 
                    "Desc Corta": first_short_description,
                    "Imagen": first_img,
                    "Desc Larga": first_long_description}

    first_d_total.update(first_d_props)

    return first_d_total


In [15]:
ls_page = [get_beer(url) for url in ls_hrefs]
df_page = pd.DataFrame(ls_page)

In [16]:
df_page

,Nombre,Precio,Desc Corta,Imagen,Desc Larga,Estilo,Origen,% Alc.,% Alc. Exacto,Volumen (cl),Tipo Fermentación,IBU,IBU Exacto,Color,Envase,Otros ingredientes,Maltas,OLD-REF,Lúpulos,Levadura
0,SanFrutos Vikingathor Boris Brew,3.35,Indian Dunkel,https://www.labirratorium.com/27941-large_defa...,"Cerveza de 8.2% ABV intensa, oscura y muy lupu...",DUNKEL,Segovia,ALTO (6-9%),8.2,33 Cl,Lager (Baja Fermentación),50-75 Amargor alto,75,Marrón,Botella,NaN,NaN,NaN,NaN,NaN
1,"Boon Oude Kriek 37,5cl",7.15,Lambic / Kriek,https://www.labirratorium.com/19351-large_defa...,Cerveza de fermentación espontánea (Lambic) de...,KRIEK,Bélgica,ALTO (6-9%),6.5,37.5 Cl,Lambic (Fermentación espontánea o salvaje),0-25 Amargor bajo,NaN,Rojiza,Botella,Cerezas Naturales,Cebada y Trigo,5412783053220,NaN,NaN
2,Störtebeker Schwarz-Bier,2.5,Cerveza negra de baja fermentación,https://www.labirratorium.com/488-large_defaul...,Cerveza negra de baja fermentación de estilo S...,SCHWARZBIER,Alemania,MEDIO (4-6%),5,50 Cl,Lager (Baja Fermentación),0-25 Amargor bajo,NaN,Negro,Botella,NaN,Cebada,$4014807364025,NaN,NaN
3,Orval,3.15,Belgian Pale Ale,https://www.labirratorium.com/385-large_defaul...,Orval es una cerveza tipo ale y es la que prin...,BELGIAN PALE ALE,Bélgica,ALTO (6-9%),6.2,33 Cl,Ale (Alta Fermentación),25-50 Amargor medio,36,Amarillo pálido,Botella,Envejecida en Barrica de Roble durante 2 Años,Cebada,54040014,"Cascade, Chinook y Mount Hood",Belgian (El Diablo)
4,Augustiner Lagerbier Hell,2.5,Lager / Helles,https://www.labirratorium.com/367-large_defaul...,Cerveza de baja fermentación de estilo Munich ...,MUNICH HELLES,Alemania,MEDIO (4-6%),5.2,50 Cl,Lager (Baja Fermentación),0-25 Amargor bajo,12,Amarillo pálido,Botella,NaN,Cebada,4105250022003,NaN,NaN
5,Schneider Aventinus Weizen-Eisbock,3,"Cerveza de color medio oscuro, aromas alcohóli...",https://www.labirratorium.com/366-large_defaul...,"Cerveza de color medio oscuro, aromas alcohóli...",NaN,Alemania,NaN,12,33,Lager (Baja Fermentación),0-25 Amargor bajo,NaN,NaN,NaN,NaN,NaN,4003669018269,NaN,NaN
6,Aecht Schlenkerla Rauchbier Weizen,2.85,None,https://www.labirratorium.com/364-large_defaul...,Rauchbier (cerveza ahumada) de trigo con 5.2% ...,RAUCHBIER,Alemania,MEDIO (4-6%),5.2,50 Cl,Ale (Alta Fermentación),0-25 Amargor bajo,20,Marrón,Botella,NaN,Trigo y Cebada,4037458000159,NaN,NaN
7,Samuel Adams Boston Lager,2.45,Cerveza de baja fermentación (Lager) de color ...,https://www.labirratorium.com/343-large_defaul...,Apariencia: color ambarino cobrizo y espuma pe...,AMERICAN LAGER,EEUU,MEDIO (4-6%),4.8,33 Cl,Samuel Adams lager,25-50 Amargor medio,30,Ámbar,Botella,NaN,Samuel Adams two-row pale malt blend y Caramel 60,5012686070005,Hallertau Mittelfrueh y Tettnang Tettnanger No...,NaN
8,Westmalle Dubbel,2.18,Dubbel,https://www.labirratorium.com/102-large_defaul...,Cerveza Trapista belga con 7% ABV de color mar...,DUBBEL,Bélgica,ALTO (6-9%),7,33 Cl,Ale (Alta Fermentación),None,24,Marrón,Botella,NaN,Cebada,5412343152332,NaN,NaN
9,Duchesse De Bourgogne,2.45,Oud Bruin/ Flanders Red Ale,https://www.labirratorium.com/12300-large_defa...,Cerveza belga de estilo Ale Roja de 6.2% ABV d...,None,Bélgica,ALTO (6-9%),6.2,25 Cl,Ale (Alta Fermentación),0-25 Amargor bajo,NaN,Ámbar,Botella,NaN,Cebada,5411364151119,NaN,NaN


In [20]:
def get_page_urls(n_page):
    URL = f"https://www.labirratorium.com/es/67-cervezas-por-estilo?page={n_page}"

    web_html = requests.get(URL).content
    soup = bs(web_html, "lxml")

    ls_href_tags = soup.find_all("a", class_="thumbnail product-thumbnail")
    ls_hrefs = [tag["href"] for tag in ls_href_tags]

    return  ls_hrefs

page = 1
ls_urls_total = []
while True:
    urls = get_page_urls(page)
    if urls != []:
        ls_urls_total.extend(urls)
        page+=1
    else:
        break

ls_urls_total

['https://www.labirratorium.com/es/inicio/831--vikingathor-.html',
 'https://www.labirratorium.com/es/cerveza-lambic/284-boon-kriek-2013.html',
 'https://www.labirratorium.com/es/cervezas-alemanas/225-stortebeker-schwarz-bier.html',
 'https://www.labirratorium.com/es/botella-33cl/199-orval.html',
 'https://www.labirratorium.com/es/cervezas-alemanas/184-augustiner-lagerbier-hell.html',
 'https://www.labirratorium.com/es/inicio/183-schneider-eisbock.html',
 'https://www.labirratorium.com/es/inicio/181-schlenkerla-rauchbier-weizen.html',
 'https://www.labirratorium.com/es/inicio/173-samuel-adams-boston-lager.html',
 'https://www.labirratorium.com/es/inicio/82-westmalle-dubbel.html',
 'https://www.labirratorium.com/es/inicio/75-duchesse-de-bourgogne.html',
 'https://www.labirratorium.com/es/inicio/61-tripel-karmeliet33.html',
 'https://www.labirratorium.com/es/inicio/21-weihenstephaner-vitus.html',
 'https://www.labirratorium.com/es/cervezas-americanas-estadounidenses/19-anchor-porter.html

In [24]:
from tqdm import tqdm

In [26]:
data_beers = []
ls_bugs = []
for i in tqdm(range(len(ls_urls_total))):
    try:
        data_beers.append(get_beer(ls_urls_total[i]))
    except:
        ls_bugs.append(ls_urls_total[i])

100%|██████████| 555/555 [05:04<00:00,  1.82it/s]


In [27]:
len(ls_bugs)

19

In [30]:
df_beers = pd.DataFrame(data_beers)
df_beers.shape

(536, 23)

In [31]:
df_beers.isnull().sum()

Nombre                  0
Precio                  0
Desc Corta             11
Imagen                  0
Desc Larga              0
Estilo                 52
Origen                168
% Alc.                156
% Alc. Exacto         109
Volumen (cl)           59
Tipo Fermentación     229
IBU                   243
IBU Exacto            355
Color                 235
Envase                185
Otros ingredientes    397
Maltas                269
OLD-REF               362
Lúpulos               361
Levadura              519
Fecha                 535
Hora                  535
Lugar                 534
dtype: int64

Ya tenemos todos los datos que queremos de la cerveza: Agrupamos todo en una lista:

### Ya sabemos obtener todos los datos que nos interesan de una cerveza, ahora tenemos que aplicar esta lógica para obtener todas las demás 

Construir un DataFrame con 

Nombre,	precio,	descripcion corta, descripción larga, ruta imagen, marca, barcode,	Estilo, Origen, % Alc., % Alc. Exacto, Volumen (Cl), Tipo Fermentación, IBU, IBU Exacto, Color, Envase